<a href="https://colab.research.google.com/github/bazinga183/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...| 2015-08-31|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live Gift Card|Digital_Video_Games| 

In [9]:
# Filter for where total_votes is >= 20
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
total_votes_df = vine_df.filter("total_votes >= 20")
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R17V95WYC9ZXDD|          1|            0|         20|   N|                Y|
|R235HEIYY15NZX|          1|            0|         26|   N|                Y|
|R3BZ4DHX2YHC4V|          1|            1|         32|   N|                Y|
| RZSC8Q4630BV1|          1|            1|         26|   N|                N|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|     

In [18]:
# Filter out where helpful_votes / total_votes >= 0.5
filtered_df = total_votes_df.filter(total_votes_df["helpful_votes"] / total_votes_df["total_votes"] >= 0.5)
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [33]:
# Filter out where there was a review included (vine == 'Y')
paid_df = filtered_df.filter(filtered_df['vine'] == 'Y')
paid_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [34]:
# Filter out where there was a review included (vine == 'N')
not_paid_df = filtered_df.filter(filtered_df['vine'] == 'N')
not_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RV4UH0OWW97K4|          5|           19|         22|   N|                N|
| RG6GT6G9WNYT1|          5|           22|         22|   N|                N|
| RFBXE3Q3GYQYO|          5|           68|         70|   N|                Y|
|R34S64NPUZOS90|          5|           23|         24|   N|                N|
|R35O3GV3HZUX8B|          5|           58|         63|   N|                N|
|R2AZAMZCEUOKQT|          1|           29|         40|   N|                Y|
|R103RUVEH5YWQ4|          1|           69|         90|   N|                Y|
|R2QNPQLXMCXOH3|          5|           35|         37|   N|                Y|
| R1JENRRBQ1YMY|          5|           35|         36|   N|                N|
|R2NS0HPQ2DD5XU|          1|           35|         45|   N|     

In [43]:
# Find the total number of votes
filtered_df.agg({'total_votes' : 'sum'}).show()

+----------------+
|sum(total_votes)|
+----------------+
|          103272|
+----------------+



In [47]:
# Total reviews
filtered_df.count()

1685

In [49]:
# Find the number of five-star reviews
five_star = filtered_df.filter(filtered_df['star_rating'] == 5)
five_star.count()

631

In [54]:
# Find which reviews were paid that gave 5 stars
five_paid = five_star.filter(five_star['vine'] == 'Y')
five_paid.count()

0

In [55]:
# Find which reviews weren't paid that gave 5 stars
five_unpaid = five_star.filter(five_star['vine'] == 'N')
five_unpaid.count()

631

In [59]:
# What percentage of the paid reviews giving 5 stars make up the 5 star reviews?
percentage_paid = five_paid.count() / five_star.count()
percentage_paid

0.0

In [60]:
# What percentage of the unpaid reviews giving 5 stars make up the 5 star reviews?
percentage_unpaid = five_unpaid.count() / five_star.count()
percentage_unpaid

1.0